In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


Importo las librerías

In [4]:
import joblib
%cd "/content/drive/My Drive/repo_tesis/src/gesto_reposo/datos"
ventanas_admitidas = joblib.load("ventanas_admitidas_50_50.pkl")
data = joblib.load("data_ventanas_admitidas_50_50.pkl")
label = joblib.load("label_ventanas_admitidas_50_50.pkl")

/content/drive/My Drive/repo_tesis/src/gesto_reposo/datos


In [5]:
!pip install lmoments3

import numpy as np
import pickle
import joblib
import pandas as pd
import sys
import matplotlib.pyplot as plt

sys.path.append("/content/drive/My Drive/repo_tesis/lib")
import functions as f

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.6 MB/s eta 0:00:00


Cálculo de features

In [6]:
nChannels = 16 # N° of Channels- numero de canales con los que quiero trabajar- el maximo es MyoArm_data['S1']['emg'].shape[1]+1

l = len(ventanas_admitidas)

zcThreshold = 0.05
sscThreshold = 0.05
rms = np.zeros(l)
zc = np.zeros(l)
ssc = np.zeros(l)
wl = np.zeros(l)
mav = np.zeros(l)
ls = np.zeros(l)
mfl = np.zeros(l)
msr = np.zeros(l)
wamp = np.zeros(l)
iav = np.zeros(l)
dasdv = np.zeros(l)
_var = np.zeros(l)
DWT1 = np.zeros(l)
DWT2 = np.zeros(l)
DWT3 = np.zeros(l)

FEATURES = pd.DataFrame()
for j in range(nChannels):
    for i in range(l):
        wl[i] = f.wl(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]])
        zc[i] = f.zc(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]], zcThreshold)
        ssc[i] = f.ssc(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]], sscThreshold)
        rms[i] = f.rms(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]])
        mav[i] = f.mav(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]])
        ls[i] = f.ls(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]])
        mfl[i] = f.mfl(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]])
        msr[i] = f.msr(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]])
        wamp[i] = f.wamp(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]])
        iav[i] = f.iav(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]])
        dasdv[i] = f.dasdv(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]])
        _var[i] = f._var(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]])
        DWT1[i],DWT2[i],DWT3[i] = f.DWT(data[:,j][ventanas_admitidas[i][0]:ventanas_admitidas[i][1]])
    FEATURES["wl_c"+str(j+1)] = wl/max(abs(wl))
    FEATURES["zc_c"+str(j+1)] = zc/max(abs(zc))
    FEATURES["ssc_c"+str(j+1)] = ssc/max(abs(ssc))
    FEATURES["rms_c"+str(j+1)] = rms/max(abs(rms))
    FEATURES["mav_c"+str(j+1)] = mav/max(abs(mav))
    FEATURES["ls_c"+str(j+1)] = ls/max(abs(ls))
    FEATURES["mfl_c"+str(j+1)] = mfl/max(abs(mfl))
    FEATURES["msr_c"+str(j+1)] = msr/max(abs(msr))
    FEATURES["wamp_c"+str(j+1)] = wamp/max(abs(min(wamp)), max(wamp))
    FEATURES["iav_c"+str(j+1)] = iav/max(abs(iav))
    FEATURES["dasdv_c"+str(j+1)] = dasdv/max(abs(dasdv))
    FEATURES["var_c"+str(j+1)] = _var/max(abs(_var))
    FEATURES["DWT1_c"+str(j+1)] = DWT1/max(abs(DWT1))
    FEATURES["DWT2_c"+str(j+1)] = DWT2/max(abs(DWT2))
    FEATURES["DWT3_c"+str(j+1)] = DWT3/max(abs(DWT3))

/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients w

Guardado de las features y los labels calculados

In [7]:
%cd "/content/drive/My Drive/repo_tesis/src/gesto_reposo/datos"

ventanas_admitidas = np.array(ventanas_admitidas)
# Data exportation
labels = {'id_ventana':ventanas_admitidas[:, 2], 'id_gesto':ventanas_admitidas[:, 3], 'subject': ventanas_admitidas[:, 4]}
labels = pd.DataFrame(labels)
joblib.dump(FEATURES, "FEATURES_W200_I50_50_50.joblib")
joblib.dump(labels, "label_W200_I50_50_50.joblib")

/content/drive/My Drive/repo_tesis/src/gesto_reposo/datos


['label_W200_I50_50_50.joblib']